# Building a RAG System from Scratch - Step by Step

**Retrieval-Augmented Generation (RAG)** is a technique that enhances Large Language Models by providing them with relevant context from a knowledge base before generating answers.

**Why RAG?**
- ✅ Reduces hallucinations by grounding answers in real data
- ✅ Enables LLMs to access up-to-date information
- ✅ Allows working with private/proprietary documents
- ✅ Can cite sources for answers

**What we'll build:**
A complete RAG system that can answer questions about a document (2024 State of the Union address)

## Step 1: Install Required Dependencies

In [ ]:
import subprocess
import sys

packages = [
    "langchain",              # Core LangChain framework
    "langchain-chroma",       # Chroma vector store integration
    "langchain-openai",       # OpenAI models integration
    "langchain-core",         # Core LangChain utilities
    "python-dotenv",          # Environment variable management
    "chromadb"                # Vector database
]

print("Installing RAG dependencies...\n")
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print(f"✓ {package}")

print("\n✅ All packages installed successfully!")

**📝 Explanation:**
We install 6 essential packages:
- **langchain**: Main framework for building LLM applications
- **langchain-chroma**: Allows us to use ChromaDB as our vector database
- **langchain-openai**: Provides OpenAI's GPT models and embeddings
- **langchain-core**: Core utilities for chains and prompts
- **python-dotenv**: Loads API keys from .env file securely
- **chromadb**: Lightweight vector database for storing document embeddings

## Step 2: Import Libraries